# RAPIDS + Plotly Dash on Paperspace ML-Driven Web Apps - Tutorial 1
## Install Extra Libraries
The RAPIDSAI container is our starting point.  However, to build this dashboard, we need some extra libraries from plot.ly and jupyter.  You'll only need to install this once per container.  Leave uncommented if this is your first run.  Otherwise comment the `!conda install` lines to save time running. 

In [1]:
!conda install -y -c conda-forge dash dash-bootstrap-components dash-html-components matplotlib plotly
!conda install -y -c plotly jupyter-dash

## Imports

In [1]:
import json
import netifaces
import matplotlib

import numpy as np

import cudf
import cupy as cp
import cupy
import cuxfilter
from cuxfilter.layouts import double_feature
from cuxfilter.charts import scatter
from cuxfilter.charts import heatmap

from bokeh import palettes
import plotly.graph_objects as go

import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State

from flask import request

#!pip install jupyter-dash
from jupyter_dash import JupyterDash

from IPython.display import display, HTML

#from flask_caching import Cache

In case you need to share your dashboard with someone else, you should get your Jupyter Lab's Token

In [2]:
import os
os.getenv("JUPYTER_TOKEN")

'2a5e79f049072d25642240c28278e066'

## Import the Data.  
This data is a pre-processed UMAP emebbeding of 65k+ cells and their gene expressions.  The pre-processing notebook can be found in "https://github.com/clara-parabricks/rapids-single-cell-examples"

In [3]:
tdf = cudf.read_csv("embeddings.csv")

In [4]:
tdf.head(10)

,Unnamed: 0,x,y,labels,ACE2,TMPRSS2,EPCAM,a_labels,t_labels,e_labels
0,0,9.503851,-8.351698,0,0.0,0.0,0.0,0,0,0
1,1,9.392367,-8.634367,0,0.0,0.0,0.0,0,0,0
2,2,9.678045,-7.977596,0,0.0,0.0,0.0,0,0,0
3,3,9.367997,-8.725381,0,0.0,0.0,0.0,0,0,0
4,4,8.550529,-8.876944,0,0.0,0.0,0.0,0,0,0
5,5,9.212639,-9.136615,0,0.0,0.0,0.0,0,0,0
6,6,8.253753,-8.168339,0,0.0,0.0,0.0,0,0,0
7,7,9.153861,-8.079064,0,0.0,0.0,0.0,0,0,0
8,8,9.355730,-7.663003,0,0.0,0.0,0.0,0,0,0
9,9,9.194642,-8.888465,0,0.0,0.0,0.0,0,0,0


In [5]:
tdf.count()

Unnamed: 0    65462
x             65462
y             65462
labels        65462
ACE2          65462
TMPRSS2       65462
EPCAM         65462
a_labels      65462
t_labels      65462
e_labels      65462
dtype: int64

## Data Description
This represents a returned UMAP dataframe from raw cell data.  We're sharing post processed data for now because the exercise is on visualizaing, not running.

- X and Y are the 2D coorindates of the UMAPembeddings. 
- Each labels are actually clusters
- ACE2, TMPRSS2 and EPCAM are the gene expressions of interest
- Each Gene expression has its own cluster

We need to be preprocess the data for Plot.ly Dash to render it.

Pro tip: if you need GPU acceleration on the client side, use `gl` after the hart type, like `scatter` to `scattergl`

## Let's make a controller!

In [6]:
def update_graph(df):
    fig = go.Figure()
    for i in range(0, len(df['labels'].unique())):
        si = str(i)
        query = 'labels == '+str(i)
        gdf = df.query(query)
        fig=fig.add_trace(
            go.Scattergl({
            'x': gdf['x'].to_array(),
            'y': gdf['y'].to_array(),
            'text': gdf['labels'].to_array(),
            'customdata': gdf['ACE2'].to_array(),
            'name': 'Label '+si,
            'mode': 'markers',
            'marker': {'size': 12}
        }))
    fig.update_layout(showlegend=True, clickmode='event+select')
    return fig

In [7]:
def update_umap_viz(df, value):
    umap_df = df[df['labels'].isin(value)]
    return update_graph(umap_df)

## Let's get that view on

In [18]:
#Import stye sheets
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
### Start the webapp

app = JupyterDash(__name__, 
                  external_stylesheets=external_stylesheets, 
                  requests_pathname_prefix='/proxy/80/') # use this for when you are on more secured instances
#app = dash.Dash(__name__, external_stylesheets=external_stylesheets) # use this is for when you control all the ports
styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

#bring in yur figure data
fig = update_graph(tdf) #here is where you pull in your cudf dataframe to populate the main graph
print(fig.layout)

#let's make your view!
app.layout = html.Div(id='site_body', children=[
        dcc.Graph(id='basic-interactions',figure=fig),
        html.Div(className='row', children=[
            html.Div([
                dcc.Markdown("""
                    **Hover Data**

                    Mouse over values in the graph.
                """),
                html.Pre(id='hover-data', style=styles['pre'])
            ], className='three columns'),


            html.Div([
                dcc.Markdown("""
                    **Click Data**

                    Click on points in the graph.
                """),
                html.Pre(id='click-data', style=styles['pre']),
            ], className='three columns'),


            html.Div([
                dcc.Markdown("""
                    **Selection Data**

                    Choose the lasso or rectangle tool in the graph's menu
                    bar and then select points in the graph.

                    Note that if `layout.clickmode = 'event+select'`, selection data also 
                    accumulates (or un-accumulates) selected data if you hold down the shift
                    button while clicking.
                """),
                html.Pre(id='selected-data', style=styles['pre']),
            ], className='three columns'),

            html.Div([
                dcc.Markdown("""
                    **Zoom and Relayout Data**

                    Click and drag on the graph to zoom or click on the zoom
                    buttons in the graph's menu bar.
                    Clicking on legend items will also fire
                    this event.
                """),
                html.Pre(id='relayout-data', style=styles['pre']),
            ], className='three columns'),
            html.Div([
                html.Button('Shutdown Dash', id='bt_close', n_clicks=0), #tried using a button
                html.Div(id='close')
            ]),
        ])
])

### this displays data of point you're hovered over
@app.callback(
    Output('hover-data', 'children'),
    [Input('basic-interactions', 'hoverData')])
def display_hover_data(hoverData):
    return json.dumps(hoverData, indent=2)

### this displays data on point that you clicked on
@app.callback(
    Output('click-data', 'children'),
    [Input('basic-interactions', 'clickData')])
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)

### this displays data of all the points you just selected
@app.callback(
    Output('selected-data', 'children'),
    [Input('basic-interactions', 'selectedData')])
def display_selected_data(selectedData):
    return json.dumps(selectedData, indent=2)

### this displays zoom information
@app.callback(
    Output('relayout-data', 'children'),
    [Input('basic-interactions', 'relayoutData')])
def display_relayout_data(relayoutData):
    return json.dumps(relayoutData, indent=2)

### this makes the botton close the dashboard
@app.callback(
    Output("site_body", "children"), 
    Input('bt_close', 'n_clicks'))
def export_current_df(export_clicks): 
    if not dash.callback_context.triggered:
        raise dash.exceptions.PreventUpdate
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()
    return 'Closed'

js = "<b style='color: red'>Please click on <a href='/proxy/80/' target='_blank'>here</a> to open the dash</b>"
display(HTML(js))

app.run_server(debug=True, use_reloader=False, port=80)

Layout({
    'clickmode': 'event+select', 'showlegend': True, 'template': '...'
})


Dash app running on http://127.0.0.1:80/proxy/80/


In [13]:
# This is a sharable version of the link (for Paperspace)
import os
nid = os.environ['PAPERSPACE_NOTEBOOK_ID']
cid = os.environ['PAPERSPACE_CLUSTER_ID']
print('https://'+nid+'.'+cid+'.paperspacegradient.com/proxy/80/')

https://nd9ecf8wlt.clg07azjl.paperspacegradient.com/proxy/80/
